# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [28]:
import numpy as np
import pandas as pd
import scipy.stats as st

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
salaries=pd.read_csv("Current_Employee_Names__Salaries__and_Position_Titles.csv")

Examine the `salaries` dataset using the `head` function below.

In [3]:
salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [8]:
salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [13]:
salaries["Salary or Hourly"].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [15]:
salaries.groupby("Department").agg({"Name":"count"})


,Name
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [86]:
#1. setting hypothesis
H0="mu of hourly wage = 30"
H1="mu of hourly wage != 30"

#2. choosing significance level
alpha=0.05

#3. sampling - not clear whether we needed to sample a certain amount out of all hourly workers OR just take all hourly workers since it's already a slice
#I went with taking all hourly workers as a sample from salaries population
h_sample=salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"]

#4. compute statistic
#I'll calculate manually to practice; then confirm with python...
mu=30
mean=h_sample.mean()
#again, not quite clear if we can just use std of the population since we have access to it? I chose to calculate sample std with ddof=1 instead and follow t distribution
s=h_sample.std(ddof=1)
n=h_sample.count()
stat=(mean-mu)/(s/np.sqrt(n))
print("Manual stat: ",stat)

#5. compute p-value - two-tailed
p_value=st.t.sf(abs(stat),n-1)*2
print("Manual p-value: ",p_value)

#computing both stat and p value through function:
print("Python calc stat: ",st.ttest_1samp(h_sample,mu)[0])
print("Python calc p-value: ",st.ttest_1samp(h_sample,mu)[1])

#6. decision
if p_value>alpha:
    print("I cannot reject the null hypothesis, which is ",H0)
else:
    print("I CAN reject the null hypothesis, so ",H1)

Manual stat:  20.619805785494183
Manual p-value:  4.3230240486245884e-92
Python calc stat:  20.6198057854942
Python calc p-value:  4.3230240486229894e-92
I CAN reject the null hypothesis, so  mu of hourly wage != 30


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [87]:
H0="mu salary <= 86000"
H1="mu salary > 86000"
mu=86000
alpha=0.05
p_sample=salaries[salaries["Department"]=="POLICE"]["Annual Salary"]
p_sample.dropna(inplace=True)
print("statistic: ",st.ttest_1samp(p_sample,mu,alternative="greater")[0])
#statistic with the same sign as the alternative hypothesis
print("p-value: ",st.ttest_1samp(p_sample,mu,alternative="greater")[1])
if p_value>alpha:
    print("I cannot reject the null hypothesis, which is ",H0)
else:
    print("I CAN reject the null hypothesis, so ",H1)

statistic:  3.081997005712994
p-value:  0.0010301701775482569
I CAN reject the null hypothesis, so  mu salary > 86000


C:\Users\valer.DESKTOP-8CM37D7\AppData\Local\Temp\ipykernel_23372\2277271498.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  p_sample.dropna(inplace=True)


Using the `crosstab` function, find the department that has the most hourly workers. 

In [73]:
pd.crosstab(salaries["Department"],salaries["Salary or Hourly"])["Hourly"].idxmax()

'STREETS & SAN'

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [89]:
H0="mu hourly >= 35"
H1="mu hourly < 35"
mu=35
alpha=0.05
h_san_sample=salaries[salaries["Department"]=="STREETS & SAN"]["Hourly Rate"]
h_san_sample.dropna(inplace=True)
print("statistic: ",st.ttest_1samp(h_san_sample,mu,alternative="less")[0])
#statistic with the same sign as the alternative hypothesis
print("p-value: ",st.ttest_1samp(h_san_sample,mu,alternative="less")[1])
if p_value>alpha:
    print("I cannot reject the null hypothesis, which is ",H0)
else:
    print("I CAN reject the null hypothesis, so ",H1)


statistic:  -9.567447887848152
p-value:  1.6689265282353859e-21
I CAN reject the null hypothesis, so  mu hourly < 35


C:\Users\valer.DESKTOP-8CM37D7\AppData\Local\Temp\ipykernel_23372\158618580.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h_san_sample.dropna(inplace=True)


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [96]:
h_sample=salaries[salaries["Salary or Hourly"]=="Hourly"]["Hourly Rate"]
mean=h_sample.mean()
c=0.95
n=h_sample.count()
s = h_sample.std(ddof=1)
st.t.interval(c,n-1,loc=mean,scale=s/np.sqrt(n))

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [97]:
mean=p_sample.mean()
c=0.95
n=p_sample.count()
s = p_sample.std(ddof=1)
st.t.interval(c,n-1,loc=mean,scale=s/np.sqrt(n))

(86177.05631531785, 86795.77269094893)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [113]:
H0="p = 0.25"
H1="p != 0.25"

alpha=0.05

from statsmodels.stats.proportion import proportions_ztest
p=0.25
success=salaries[salaries["Salary or Hourly"]=="Hourly"]["Name"].count()
n=salaries["Name"].count()

p_value=proportions_ztest(success, n, p)
print(p_value)
if p_value[1]>alpha:
    print("I cannot reject the null hypothesis, which is ",H0)
else:
    print("I CAN reject the null hypothesis, so ",H1)


(-3.5099964213703005, 0.0004481127249057967)
I CAN reject the null hypothesis, so  p != 0.25
